In [1]:
import cv2
import dlib
import numpy as np
from imutils import face_utils
from scipy.spatial import distance

# Dlibの学習済みモデルファイルを読み込み
face_parts_detector = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
detector = dlib.get_frontal_face_detector()

# クマの左耳と右耳、鼻の画像を読み込み
left_bear_ear = cv2.imread("bear/left_bear_ear.png", cv2.IMREAD_UNCHANGED)
right_bear_ear = cv2.imread("bear/right_bear_ear.png", cv2.IMREAD_UNCHANGED)
bear_nose = cv2.imread("bear/bear_nose.png", cv2.IMREAD_UNCHANGED)

# クッキーの画像を読み込み
cookie_img = cv2.imread("bear/cookie.png", cv2.IMREAD_UNCHANGED)  # 透過PNG形式

# 透過PNG画像の合成関数
def overlay_image_alpha(img, overlay, pos, scale=1.0):
    # オーバーレイ画像のサイズをスケールで調整
    overlay = cv2.resize(overlay, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)
    x, y = pos
    h, w = overlay.shape[0], overlay.shape[1]

    # 境界チェック（オーバーレイ画像が元の画像をはみ出ないようにする）
    if x + w > img.shape[1]:
        w = img.shape[1] - x
    if y + h > img.shape[0]:
        h = img.shape[0] - y
    if x < 0:
        x = 0
    if y < 0:
        y = 0

    # マスクと逆マスクを作成
    if overlay.shape[2] == 4:  # 透過チャンネルがある場合
        alpha_mask = overlay[:h, :w, 3] / 255.0
        alpha_image = 1.0 - alpha_mask

        # オーバーレイ部分を合成
        for c in range(0, 3):
            img[y:y+h, x:x+w, c] = (alpha_image * img[y:y+h, x:x+w, c] + alpha_mask * overlay[:h, :w, c])

    return img

# 口の開閉を判定する関数
def calc_mouth(mouth):
    # 口のアスペクト比（MAR）を計算
    A = distance.euclidean(mouth[2], mouth[10])  # 上唇と下唇の縦距離
    B = distance.euclidean(mouth[4], mouth[8])   # 上唇と下唇の縦距離
    C = distance.euclidean(mouth[0], mouth[6])   # 左右の横距離
    MAR = (A + B) / (2.0 * C)
    return round(MAR, 3)

# カメラデバイスをオープン
cap = cv2.VideoCapture(0)

###################
# メイン処理       #
###################
while True:
    ret, image = cap.read()
    if not ret:
        break

    # グレースケールに変換
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # 顔検出
    rects = detector(gray, 1)

    for rect in rects:
        # ランドマークを取得
        shape = face_parts_detector(gray, rect)
        shape = face_utils.shape_to_np(shape)

        # 顔のランドマークの位置を取得
        left_eye = shape[36]  # 左目の外側
        right_eye = shape[45]  # 右目の外側
        nose = shape[30]  # 鼻の位置
        chin = shape[8]  # 顎のランドマーク

        # 顔の幅と高さを基に耳の位置を調整
        face_width = np.linalg.norm(left_eye - right_eye)  # 両目間の距離
        face_height = np.linalg.norm(nose - chin)  # 顔の高さ

        # 左耳のX座標とY座標を計算
        left_ear_x = int(left_eye[0]) - int(face_width * 0.5)  # 左耳は左目の外側から少し左に配置
        left_ear_y = int(left_eye[1]) - int(face_height * 1.2)  # 左耳のY位置は顔の上部に調整

        # 右耳のX座標とY座標を計算
        right_ear_x = int(right_eye[0]) + int(face_width * -0.2)  # 右耳は右目の外側から少し右に配置
        right_ear_y = int(right_eye[1]) - int(face_height * 1.25)  # 右耳のY位置も顔の上部に調整

        # 左耳を合成
        image = overlay_image_alpha(image, left_bear_ear, (left_ear_x, left_ear_y), scale=face_width / 200)

        # 右耳を合成
        image = overlay_image_alpha(image, right_bear_ear, (right_ear_x, right_ear_y), scale=face_width / 200)

        # 鼻の位置にクマの鼻を合成
        nose_x = nose[0] - 58  # 鼻の画像のX位置調整
        nose_y = nose[1] - 70  # 鼻の画像のY位置調整
        image = overlay_image_alpha(image, bear_nose, (nose_x, nose_y), scale=0.3)

        # 口のランドマーク（48番から67番の点）を取得
        mouth = shape[48:68]

        # 口のアスペクト比（MAR）を計算
        MAR = calc_mouth(mouth)

        # 口のMARが閾値を超えているかチェック（0.6を閾値として設定）
        if MAR > 0.6:
            # クッキーを口の位置に合成
            mouth_center = mouth.mean(axis=0).astype(int)  # 口の中心座標を計算
            
            # クッキーの縦位置を調整（唇の位置を参考に）
            lip_distance = distance.euclidean(mouth[3], mouth[9])  # 上唇と下唇の距離
            image = overlay_image_alpha(image, cookie_img, (mouth_center[0] - 20, mouth_center[1] - int(lip_distance * 0.5)), scale=0.5)

    # 画像を表示
    cv2.imshow('Frame', image)

    # 'q'キーで終了
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 終了処理
cap.release()
cv2.destroyAllWindows()
